# LangChain Expression Language (LCEL)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

**Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

**Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

**Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

**Seamless LangSmith Tracing Integration** As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.

<div style="text-align:center;"><img src="imgs/lcel.PNG" width="540" height="600"/></div>

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
#!pip install pydantic==1.10.8

In [2]:
# Import components that will be composed together
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser # convert chat message to string

## Simple Chain

Let's create a simple chain:

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "ChatGPT"})

"Why did ChatGPT go to therapy?\nBecause it couldn't stop talking in binary code!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
# Create a vectorstore from documents and embeddings
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"], # documents
    embedding=OpenAIEmbeddings()                              # embedding object
)
# Initialize a retriever
retriever = vectorstore.as_retriever()

In [8]:
# Retrieve documents similar to a query
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

> It returns both docs because it is a simple example with 2 docs in vector store!

In [9]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain.schema.runnable import RunnableMap

In [13]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [14]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [15]:
# Closer look at RunnableMap to see what happens behind the scenes
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [16]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind
One of the other things we can do with `Runnables` that we can bind parameters to them.
Let's do that with an example with OpenAI Functions:

In [17]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

Under the hood, when the LLM model is called, it will pass any paramters in ``bind`` along to the **invokation**.

In [19]:
runnable = prompt | model

In [20]:
runnable.invoke({"input": "what is the weather in Düsseldorf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

In [21]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [22]:
model = model.bind(functions=functions)

In [23]:
runnable = prompt | model

In [24]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "patriots"\n}'}})

## Fallbacks

There are many possible points of failure in an LLM application, whether that be issues with LLM API's, poor model outputs, issues with other integrations, etc. Fallbacks help you gracefully handle and isolate these issues.

Crucially, fallbacks can be applied not only on the LLM level but on the whole runnable level.

Let's see how fallbacks work using a chain that would fail:

In [25]:
from langchain.llms import OpenAI # Different from ChatOpenAI
import json

In [34]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
)
# Create a chain that first runs the simple model, then parses the output as JSON
bad_chain = simple_model | json.loads

This chain will break if the output is not valid JSON file

In [35]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [36]:
# Simple model works!
simple_model.invoke(challenge)

'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

Note: The next line is expected to fail.

In [37]:
# But the chain fails
bad_chain.invoke(challenge)

JSONDecodeError: Extra data: line 5 column 1 (char 103)

Let's use new models from OpenAI `ChatOpenAI` and define new chain

In [38]:
model = ChatOpenAI(temperature=0)
good_chain = model | StrOutputParser() | json.loads

In [39]:
good_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

Create new chain, where with ``good_chain`` will be invoked if the ``bad_chain`` fails

In [40]:
final_chain = bad_chain.with_fallbacks([good_chain])

In [41]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the walls, a secret ballet'}}

## Interface

To make it as easy as possible to create custom chains, we've implemented a ["Runnable"](https://api.python.langchain.com/en/latest/schema/langchain.schema.runnable.base.Runnable.html#langchain.schema.runnable.base.Runnable) protocol. The ``Runnable`` protocol is implemented for most components. This is a standard interface, which makes it easy to define custom chains as well as invoke them in a standard way. 

The standard interface includes (invoked in **sync** way, i.e., executed in **parallel**):
 
* ``invoke``: call the chain on an input
* ``batch``: call the chain on a list of inputs
* ``stream``: stream back chunks of the response (stream the result instead of waiting the full ouput one time -> useful for apps)

In [42]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [43]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [44]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why did the frog take the bus to work?\nBecause his car got toad!']

These also have corresponding **async** methods:

* ``astream``: stream back chunks of the response async
* ``ainvoke``: call the chain on an input async
* ``abatch``: call the chain on a list of inputs async

In [45]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?
 


Because
 they
 have
 bear
 feet
!



In [46]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes? \n\nBecause they have bear feet!"